In [1]:
import os
# Changing the working directory
os.chdir('../..')
os.getcwd()

'/Users/javiermunoz/Universidad/MasterDSyBD/TFM/TFM_DSBD'

In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc("figure", figsize=(15, 5))

import numpy as np
import pandas as pd

In [3]:
gpu_devices

[]

In [4]:
leaderboard = pd.read_csv("data/leaderboard.csv")
leaderboard

,model_type,model_name,train_acc,val_acc,test_acc,train_loss,val_loss,test_loss,epoch,path_to_model,subset_size
0,mediapipe,mp,0.95,0.92,0.93,0.22,0.28,0.28,77.0,src/mediapipe/bestmodels/best_mp_77_0.92_model.h5,10.0
1,convlstm,simple,0.96,0.93,0.93,0.15,0.20,0.19,39.0,src/convlstm/bestmodels/best_simple_39_0.93_mo...,10.0
2,convlstm,stateless,0.93,0.95,0.91,0.22,0.19,0.19,22.0,src/convlstm/bestmodels/best_stateless_22_0.95...,10.0
3,3DCNN,3dcnn,0.92,0.90,0.87,0.35,0.39,0.42,63.0,src/3DCNN/bestmodels/best_3dcnn_63_0.90_model.h5,10.0


In [5]:
path_to_test = leaderboard['path_to_model'][0]

model_to_test = tf.keras.models.load_model(path_to_test)
model_to_test.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 20, 128)           916992    
                                                                 
 dropout (Dropout)           (None, 20, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 20, 256)           394240    
                                                                 
 dropout_1 (Dropout)         (None, 20, 256)           0         
                                                                 
 lstm_2 (LSTM)               (None, 20, 128)           197120    
                                                                 
 dropout_2 (Dropout)         (None, 20, 128)           0         
                                                                 
 lstm_3 (LSTM)               (None, 64)                4

In [6]:
from src.utils import load_dict
subset = 10
labels=load_dict(f'data/subset_{subset}_lsa_64/pickl_files/labels_map.pkl')
labels

{'Barbecue': 0,
 'Birthday': 1,
 'Buy': 2,
 'Chewing-gum': 3,
 'Coin': 4,
 'Milk': 5,
 'Mock': 6,
 'Realize': 7,
 'Sweet milk': 8,
 'To land': 9}

In [7]:
import mediapipe as mp
from numpy.random import default_rng
from src.config import LM_PER_VIDEO
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils 

rng = default_rng()

def get_landmarks(frames: list):
    
    landmarks_to_model=[]
    extracted_n_lm_idx = sorted(rng.choice(len(frames), size=LM_PER_VIDEO, replace=False))
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for i in range(len(frames)):
            if i in extracted_n_lm_idx:
                results = holistic.process(frames[i])
                if results is None:
                    print("Something is wrong w/ mediapipe... Exiting.")
                    return None
                
                landmarks_to_model.append(results)
    
    return np.array(landmarks_to_model)


In [8]:
import cv2

cap = cv2.VideoCapture(0)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

frames_to_analyze=[]
count=1 
ret=True
while ret:
    ret, frame = cap.read()
    frames_to_analyze.append(frame)
    if count % 100 == 0:
        landmarks_to_model = get_landmarks(frames_to_analyze)
        if landmarks_to_model is None:
            exit()
        logits = model_to_test.predict(np.expand_dims(landmarks_to_model, axis=0))
        pred_label = list(labels.values()).index(np.argmax(logits))
        cv2.putText(frame, pred_label)

    count += 1
    # Show to screen
    cv2.imshow('OpenCV Feed', frame)

    # Break gracefully
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


(20,)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type type).